In [1]:
import bz2
import nltk
import numpy as np
import json
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/leo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [124]:
train_file = bz2.BZ2File('../input_data/train.json.bz2')
test_file = bz2.BZ2File('../input_data/test.json.bz2')
train_file = json.load(train_file)
test_file = json.load(test_file)
print(len(train_file))
print(len(test_file))

615243
126152


In [125]:
# Preprocess data ...

link_treshold = 3
valid_links = Counter([next(iter(sentence.values())) for sentence in train_file if next(iter(sentence.values())) is not None])
valid_links = set([link for link,frequence in valid_links.items() if frequence >= link_treshold])

train_links = []
for i, sentence in enumerate(train_file):
    sentence, link = next(iter(sentence.items()))
    train_file[i] = sentence.lower().strip()
    if link in valid_links:
        train_file[i] = '_' + train_file[i] + '_'
        train_links.append(link)
        
test_links = []
for i, sentence in enumerate(test_file):
    sentence, link = next(iter(sentence.items()))
    test_file[i] = sentence.lower().strip()
    if link in valid_links:
        test_file[i] = '_' + test_file[i] + '_'
        test_links.append(link)

train_file = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(" ".join(train_file))]
test_file = [nltk.word_tokenize(sentence) for sentence in nltk.sent_tokenize(" ".join(test_file))]
train_sentences = [[word.replace('_','') for word in sentence] for sentence in train_file]
test_sentences = [[word.replace('_','') for word in sentence] for sentence in test_file]

vocabulary = Counter([word for sentence in train_sentences for word in sentence for char in word])
vocabulary = ['_PAD','_UNK'] + sorted(vocabulary, key=vocabulary.get, reverse=True)
word2idx = {w:i for i,w in enumerate(vocabulary)}
idx2word = {i:w for i,w in enumerate(vocabulary)}

In [121]:
# Encode words as integers ...

for i, sentence in enumerate(train_sentences):
    train_sentences[i] = [word2idx[word] if word in word2idx else 1 for word in sentence]

['_PAD', '_UNK', 'the', 'and', 'of', ',', 'in', '.', 'to', 'american', 'that', 'with', 'was', 'for', 'a', 'is', 'from', 'as', 'which', 'his']
